[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tuanh118/uw-fsdl-bert-transfer-translation/blob/add-collab-notebook/Notebook.ipynb)

## Set up the Collab VM and import libraries.


In [1]:
# Hack to reference other files in the repo in Collab.
!git clone https://github.com/tuanh118/uw-fsdl-bert-transfer-translation
import sys
sys.path.append('./uw-fsdl-bert-transfer-translation')

# Install required packages.
!pip install transformers

%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from transformers import *
from sklearn.model_selection import train_test_split

import io
import os
import time

from CombinedBertTransformerModel import *

Cloning into 'uw-fsdl-bert-transfer-translation'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 59 (delta 19), reused 38 (delta 9), pack-reused 0
Unpacking objects: 100% (59/59), done.
     |████████████████████████████████| 665kB 9.1MB/s 
     |████████████████████████████████| 1.1MB 56.7MB/s 
     |████████████████████████████████| 890kB 53.6MB/s 
     |████████████████████████████████| 3.8MB 36.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=0a67e746e0bfb89b2ec34be3cb8fe0f096226c23f81404c6ec0446a9c44200bf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## Data retrieval

In [3]:
# Download the EuroParl French-English corpus.
path_to_fr_en_tar = tf.keras.utils.get_file('fr-en.tgz', origin='https://www.statmt.org/europarl/v7/fr-en.tgz', extract=True)
path_to_fr_en_en_file = os.path.dirname(path_to_fr_en_tar) + "/europarl-v7.fr-en.en"
path_to_fr_en_fr_file = os.path.dirname(path_to_fr_en_tar) + "/europarl-v7.fr-en.fr"


202719232/202718517 [==============================] - 47s 0us/step


## Data processing

In [4]:
# Sets up a BERT tokenizer.
def instantiate_tokenizer():
    return BertTokenizer.from_pretrained('bert-base-uncased')

# Given a path to a text file, load and tokenize each line using the provided tokenizer, then convert each token to an ID and pad all lines to have length max_tokens.
def load_dataset(language_path, tokenizer, num_examples=None, max_tokens=500):
    # Read the data.
    lines = io.open(language_path, encoding='UTF-8').read().strip().splitlines()[:num_examples]

    # Tokenize and add the special start token.
    tokenized_lines = [ ['[CLS]'] + tokenizer.tokenize(line)[:max_tokens-1] + ['[SEP]'] for line in lines ]
    
    # Convert tokens to IDs.
    ids = [ tokenizer.convert_tokens_to_ids(tokenized_line) for tokenized_line in tokenized_lines ]

    # Generate padding masks and segment IDs. These have the same length as the ID sequences after padding.
    # Padding mask is 1 where there is an actual ID and 0 where there is padding. Segment ID is always 0.
    masks = [ [1] * len(tokenized_line) for tokenized_line in tokenized_lines ]
    segments = [ [] for tokenized_line in tokenized_lines ]

    # Pad all ID sequences to the maximum length with zeroes.
    ids = tf.keras.preprocessing.sequence.pad_sequences(ids, maxlen=max_tokens, truncating="post", padding="post", dtype="int")
    masks = tf.keras.preprocessing.sequence.pad_sequences(masks, maxlen=max_tokens, truncating="post", padding="post", dtype="int")
    segments = tf.keras.preprocessing.sequence.pad_sequences(segments, maxlen=max_tokens, truncating="post", padding="post", dtype="int")

    return ids, masks, segments

num_examples = 300
max_tokens = 50
tokenizer = instantiate_tokenizer()
input_tensor, masks, segments = load_dataset(path_to_fr_en_en_file, tokenizer, num_examples, max_tokens)
target_tensor, _, _ = load_dataset(path_to_fr_en_fr_file, tokenizer, num_examples, max_tokens)

# Split the data into training and validation sets.  No test set for now since we're just experimenting.
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Do some printing to show that the processing worked.
def convert(tokenizer, tensor):
    for t in tensor:
        if t != 0:
            print ("%d ----> %s" % (t, tokenizer.ids_to_tokens[t]))

print("ID to token mapping for first training example (input)")
convert(tokenizer, input_tensor_train[0])
print()
print("ID to token mapping for first training example (target)")
convert(tokenizer, target_tensor_train[0])

BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train) // BATCH_SIZE
embedding_dim = 32
units = 32
vocab_size = len(tokenizer.vocab)

train_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(len(input_tensor_train)).batch(BATCH_SIZE, drop_remainder=True)
validation_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val)).shuffle(len(input_tensor_val)).batch(BATCH_SIZE, drop_remainder=True)


ID to token mapping for first training example (input)
101 ----> [CLS]
2035 ----> all
1997 ----> of
2023 ----> this
2003 ----> is
1999 ----> in
10388 ----> accordance
2007 ----> with
1996 ----> the
6958 ----> principle
1997 ----> of
4942 ----> sub
5332 ----> ##si
9032 ----> ##dia
15780 ----> ##rity
1998 ----> and
2003 ----> is
3568 ----> therefore
2000 ----> to
2022 ----> be
6551 ----> greatly
10979 ----> welcomed
1012 ----> .
102 ----> [SEP]

ID to token mapping for first training example (target)
101 ----> [CLS]
2000 ----> to
4904 ----> ##ut
8292 ----> ce
6895 ----> ##ci
9193 ----> doi
2102 ----> ##t
3802 ----> et
2890 ----> ##re
2112 ----> part
2594 ----> ##ic
15859 ----> ##uli
7869 ----> ##ere
3672 ----> ##ment
16183 ----> sal
5657 ----> ##ue
4372 ----> en
2744 ----> term
2229 ----> ##es
2139 ----> de
4942 ----> sub
5332 ----> ##si
9032 ----> ##dia
17625 ----> ##rite
1012 ----> .
102 ----> [SEP]


## Model preparation

In [5]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

print(train_dataset.as_numpy_iterator().next()[0][0].shape)

model = CombinedBertTransformerModel(
    max_tokens=max_tokens,
    vocab_size=vocab_size,
    num_layers=2,
    units=32,
    d_model=32,
    num_heads=2,
    dropout=0.2,
    padding_label=0
)
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
model.summary()

(50,)



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tokenized_output_sentence (Inpu [(None, 50)]         0                                            
__________________________________________________________________________________________________
tokenized_input_sentence (Input [(None, 50)]         0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 50, 768), (N 109482240   tokenized_input_sentence[0][0]   
__________________________________________________________________________________________________
look_ahead_mask (Lambda)        (None, 1, None, 50)  0           tokenized_output_sentence[0][0]  
_____________________________________________________________________________________________

## Model training

In [6]:
# Train and evaluate the model using tf.keras.Model.fit()
# TODO This doesn't work yet.
history = model.fit(
    train_dataset,
    epochs=2,
    steps_per_epoch=115,
    validation_data=validation_dataset,
    validation_steps=7
)

Epoch 1/2


AssertionError: ignored